In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import os
from itertools import product

from urban_growth.model import *
from urban_growth.simulator import *
from urban_growth.estimator import *

%matplotlib inline

# Construct the Control Frame

In [2]:
files = os.listdir('data')
maps = [name for name in files if 'mask' not in name]
control_df = pd.DataFrame({'file_name' : maps} )

control_df['city'] = control_df.file_name.str[0:3]
control_df['year_begin'] = pd.to_numeric(control_df.file_name.str[3:7])
control_df = control_df[control_df.year_begin < 2011]

def get_year_end(year):
    if year in [1990, 2000]:
        return year + 10
    else:
        return year + 5
    
v_gye = np.vectorize(get_year_end)

control_df['year_end'] = v_gye(control_df.year_begin)

control_df = control_df.drop('file_name', 1)

# Define Params

In [3]:
# kernel params, probably won't loop over these
L = 20 
base_val = .4
unit = 3.333 # pixels per km
K = distance_kernel(L, base_val, unit)

# Data processing params, no need to touch
N_pix = 100

# initial parameters, might need to loop over these

par_0 = {'alpha' : np.array([100, 30]),
        'gamma' : np.array([1.2, 2.5]),
        'beta'  : -25}

# thresholding params, will loop over these
sigma = np.linspace(0, 5, 26)
t     = np.linspace(0, .5, 10)

# Run Grid Search

In [9]:
def grid_search(city, year_begin, year_end, sigma, t, par_0):

    # get and clean data for given city and years
    M0  = np.loadtxt('data/' + city + str(year_begin) + '.csv', dtype=float, delimiter=',')
    M1  = np.loadtxt('data/' + city + str(year_end) + '.csv',   dtype=float, delimiter=',')
    geo = np.loadtxt('data/' + city + '_mask.csv',   dtype=float, delimiter=',')

    geo = 1 - geo
    M0 = M0 * geo
    M1 = M1 * geo

    M1  = np.maximum(M0, M1)
    
    # initialize lists
    
    city_vec       = []
    year_begin_vec = []
    year_end_vec   = []
    sig_vec        = []
    t_vec          = []
    par_0_vec      = []
    par_vec        = []
    ll_vec         = []
    cov_vec        = []
    N_eff_vec      = []
    n_pars_vec     = []
    AIC_vec        = []
    rate_vec       = [] 
    bg_rate_vec    = []
        
        # computations
    for sig,tau in product(sigma, t):
        e = estimator(M0 = M0, geo = geo,  N_pix = N_pix, sigma = sig, t = tau)
        res = e.ml(M1, K, par_0, opts = {'disp' : True}, use_grad = True)
        
        if t == 0:
            n_pars = 4
        else:
            n_pars = 7
        
        ll = e.log_likelihood(K = K, M1 = M1, pars = res[0])
        
        N_eff = ((1 - M0) * geo * N_pix).sum()
        settlement_rate = (M1 - M0).sum() * N_pix / N_eff
        bg_rate = expit(res[0]['beta'])
        
        # updates to storage lists
        city_vec.append(city)
        year_begin_vec.append(year_begin)
        year_end_vec.append(year_end)
        sig_vec.append(sig)
        t_vec.append(tau)
        par_0_vec.append(par_0)
        par_vec.append(res[0])
        ll_vec.append(ll)
        cov_vec.append(res[2])
        N_eff_vec.append(N_eff)
        n_pars_vec.append(n_pars)
        AIC_vec.append(2 * (n_pars - ll))
        rate_vec.append(settlement_rate)
        bg_rate_vec.append(bg_rate)
        
    df = pd.DataFrame({
        'city'       : city_vec,
        'year_begin' : year_begin_vec,
        'year_end'   : year_end_vec,
        'sigma'      : sig_vec,
        't'          : t_vec,
        'par_0'      : par_0_vec,
        'par'        : par_vec,
        'll'         : ll_vec,
        'cov'        : cov_vec,
        'N_eff'      : N_eff_vec,
        'n_pars'     : n_pars_vec,
        'AIC'        : AIC_vec,
        'settlement_rate' : rate_vec,
        'bg_rate' : bg_rate_vec
    })
    
    return df

In [10]:
test = grid_search('SHA', 1990, 2000, [0, 1], [.1, .5],  par_0) 

Optimization terminated successfully.
         Current function value: 24.266051
         Iterations: 55
         Function evaluations: 88
         Gradient evaluations: 88
Optimization terminated successfully.
         Current function value: 24.275317
         Iterations: 77
         Function evaluations: 96
         Gradient evaluations: 96


urban_growth/estimator.py:32: RuntimeWarning: divide by zero encountered in divide
  grad    =   self.N_pix*((M1-self.M0)/p-(1-M1)/(1-p))*grad
urban_growth/estimator.py:32: RuntimeWarning: invalid value encountered in multiply
  grad    =   self.N_pix*((M1-self.M0)/p-(1-M1)/(1-p))*grad


Optimization terminated successfully.
         Current function value: 24.228357
         Iterations: 45
         Function evaluations: 62
         Gradient evaluations: 60
Optimization terminated successfully.
         Current function value: 24.342142
         Iterations: 68
         Function evaluations: 93
         Gradient evaluations: 93


In [11]:
test

,AIC,N_eff,bg_rate,city,cov,ll,n_pars,par,par_0,settlement_rate,sigma,t,year_begin,year_end
0,3.736064e+06,14557623.0,0.037848,SHA,"[[17053.764999, 153.054086162, -136.286222704,...",-1.868025e+06,7,"{u'alpha': [41.607569304, 4.68040574644], u'be...","{u'alpha': [100, 30], u'beta': -25, u'gamma': ...",0.097764,0,0.1,1990,2000
1,3.739398e+06,14557623.0,0.056788,SHA,"[[7.92230970477, -0.686747393768, -39.79482235...",-1.869692e+06,7,"{u'alpha': [3.51204952871, 4.18842998456], u'b...","{u'alpha': [100, 30], u'beta': -25, u'gamma': ...",0.097764,0,0.5,1990,2000
2,3.722500e+06,14557623.0,0.032295,SHA,"[[3031.4150545, 137.284538269, -25.5195801163,...",-1.861243e+06,7,"{u'alpha': [34.2126156327, 5.5109861479], u'be...","{u'alpha': [100, 30], u'beta': -25, u'gamma': ...",0.097764,1,0.1,1990,2000
3,3.763445e+06,14557623.0,0.057797,SHA,"[[49.2986041154, -2.10672610801, -16.139300328...",-1.881716e+06,7,"{u'alpha': [3.63023115696, 4.27284967982], u'b...","{u'alpha': [100, 30], u'beta': -25, u'gamma': ...",0.097764,1,0.5,1990,2000
